# Proyecto 4
- #### Miguel González

Para ver el video del proyecto haga clic [aquí.](https://iteso01-my.sharepoint.com/:v:/g/personal/is722298_iteso_mx/EbS-ziHBxoFDjoSzGA4AlyoBXOExck_TzGPe-AkQcc95aw?e=mFpjcU "Link al video en Onedrive")

## Clasificación de vengadores  
Se buscará clasificar los vengadores a partir de un dataset que contiene las imágenes de cada uno de los integrantes en el equipo. En total el dataset contiene 9 personajes diferentes cada uno con mínimo 79 y un máximo de 177 imágenes por categoría. Dichas imagenes fueron procesadas con el algorítmo de deep learning de inception V3 desde Orange.

### Importar librerías

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score as cv

### Lectura de datos

El archivo csv contiene un total de 2048 features por cada imagen, además de esto, cada registro tiene la categoría a la que pertenece, esta siendo el personaje que se encuentra de la imágen. En total existen 1208 registros de personaje.

In [2]:
imageData = pd.read_csv('C:\\Users\\gonza\\AprendizajeMaquina\\Proj-4\\combinedDataOGChar.csv')
imageData.head()

,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,...,n2039,n2040,n2041,n2042,n2043,n2044,n2045,n2046,n2047,category
0,0.395291,0.250624,0.099512,0.268821,0.091371,0.432410,0.327322,0.357707,0.319891,0.181302,...,0.429419,0.166420,0.456801,0.553125,0.702891,0.160196,0.345281,0.099554,0.030428,BlackWidow
1,0.161620,0.669297,0.161754,1.092749,0.314478,0.739942,0.289364,0.257756,0.456076,0.945528,...,0.125760,0.841949,0.751705,0.330969,0.945445,0.087484,0.420044,0.073521,0.496550,BlackWidow
2,0.613029,0.272452,0.228112,0.211418,0.018076,0.175503,0.173159,0.360020,0.467546,1.045792,...,0.954695,0.156586,0.405534,0.447233,0.147782,0.820623,0.269167,0.803643,0.295057,BlackWidow
3,0.435276,0.061578,0.049443,0.273972,0.198103,0.364443,0.403887,0.446709,0.077413,0.206818,...,0.122626,0.519245,0.480481,0.564943,0.131951,0.126217,0.178021,0.930107,0.740767,BlackWidow
4,0.780187,0.411210,1.116514,0.303062,0.018254,0.100198,0.461828,0.524668,0.273504,0.419966,...,0.550560,0.189530,0.149799,0.249432,0.587155,0.180132,0.311663,0.175716,0.067949,BlackWidow


Podemos observar los diferentes personajes que hay dentro del dataset

In [3]:
list(dict.fromkeys(imageData['category']))

['BlackWidow',
 'CaptainAmerica',
 'DrStrange',
 'Hulk',
 'Ironman',
 'Loki',
 'Spiderman',
 'Thanos',
 'Thor']

In [4]:
imageData.shape

(1208, 2049)

#### Separación del dataset
A partir de los datos generamos nuestras variables de entrenamiento y el target. Para la repartición entre datos de entrenamiento y validación habrá una relación de 70-30 por cada uno de los personajes; el 70% es sobre el total de imágenes que existen en cada una de las categorías.

In [5]:
img_X, img_Y = imageData.drop(['category'], axis=1), imageData['category']
img_X_train, img_X_test, img_Y_train, img_Y_test = train_test_split(img_X, img_Y, test_size=0.3, random_state = 1, stratify=img_Y)
img_X_train.shape

(845, 2048)

### Modelos

Para el proyecto se utilizarán los modelos de:
- Random Forest
- SVM
- Neural Network

Cada uno de los modelos tendrá 2 variaciones para así obtener diferentes resultados utilizando el mismo modelo y poder observar los efectos que el parámetro en cuestión puede llegar a tener.

#### GridSearch
Apartir de modificaciones dentro de sus parámetros se buscarán los modelos que tengan los mejores resultados

In [ ]:
parameters = {'n_estimators':[150, 300, 500, 700, 900, 1000, 1250, 10000], 'criterion':('gini', 'entropy'), 
              'max_features':('auto', 'sqrt', 'log2')}
forest = RandomForestClassifier()
clfForest = GridSearchCV(forest, parameters)
clfForest.fit(img_X_train, img_Y_train)

In [ ]:
pd.concat([pd.DataFrame(clfForest.cv_results_["params"]), pd.DataFrame(clfForest.cv_results_["rank_test_score"]),
           pd.DataFrame(clfForest.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1).sort_values(0)

In [ ]:
parameters = {'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1, 'scale', 'auto'],
              'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'shrinking':[True, False], 
              'probability':[True,False], 'decision_function_shape':['ovo', 'ovr']}
svc = SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(img_X_train, img_Y_train)

In [ ]:
pd.concat([pd.DataFrame(clf.cv_results_["params"]),
           pd.DataFrame(clf.cv_results_["rank_test_score"]), pd.DataFrame(clf.cv_results_["mean_test_score"], 
           columns=["Accuracy"])],axis=1).sort_values(0)

In [ ]:
parameters = {'max_iter':[150,700,500,300,1000],'hidden_layer_sizes':[100,50,200,500],'solver':['lbfgs','sgd','adam'], 
             'activation':['identity', 'logistic', 'tanh', 'relu'], 'learning_rate':['constant', 'invscaling', 'adaptive']}
mlp = MLPClassifier()
clfMlp = GridSearchCV(mlp, parameters)
clfMlp.fit(img_X_train, img_Y_train)

In [ ]:
pd.concat([pd.DataFrame(clfMlp.cv_results_["params"]),pd.DataFrame(clfMlp.cv_results_["rank_test_score"]),pd.DataFrame(clfMlp.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1).sort_values(0)

#### Creación de los modelos

Serán creados los modelos a partir de los resultados del grid search, utilizando los parámetros que obtuvieron mejor puntaje

In [6]:
img_forest_v1 = RandomForestClassifier(criterion='entropy', max_features='sqrt', n_estimators=900)

In [7]:
img_forest_v2 = RandomForestClassifier(criterion='gini', max_features='sqrt', n_estimators=700)

In [8]:
img_svc_v1 = SVC(kernel="rbf", C=100, gamma='scale', decision_function_shape='ovo',probability=True, shrinking=False)

In [9]:
img_svc_v2 = SVC(kernel="rbf", C=100, gamma='scale', decision_function_shape='ovr',probability=False, shrinking=True)

In [10]:
img_mlp_v1 = MLPClassifier(hidden_layer_sizes=100, max_iter=700, solver='adam', activation='logistic', learning_rate = 'constant')

In [11]:
img_mlp_v2 = MLPClassifier(hidden_layer_sizes=50, max_iter=1000, solver='adam')

#### Entrenamiento
Le hes alimentado los datos y los targets de entrenamiento a cada uno de los modelos

In [12]:
img_forest_v1.fit(img_X_train, img_Y_train)

RandomForestClassifier(criterion='entropy', max_features='sqrt',
                       n_estimators=900)

In [13]:
img_forest_v2.fit(img_X_train, img_Y_train)

RandomForestClassifier(max_features='sqrt', n_estimators=700)

In [14]:
img_svc_v1.fit(img_X_train, img_Y_train)

SVC(C=100, decision_function_shape='ovo', probability=True, shrinking=False)

In [15]:
img_svc_v2.fit(img_X_train, img_Y_train)

SVC(C=100)

In [16]:
img_mlp_v1.fit(img_X_train, img_Y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=100, max_iter=700)

In [17]:
img_mlp_v2.fit(img_X_train, img_Y_train)

MLPClassifier(hidden_layer_sizes=50, max_iter=1000)

#### Validación

En este apartado generamos las predicciones de cada uno de los modelos con los los datos de validación

In [18]:
Y_pred_forest_v1 = img_forest_v1.predict(img_X_test)
Y_pred_forest_v2 = img_forest_v2.predict(img_X_test)

Y_pred_mlp_v1 = img_mlp_v1.predict(img_X_test)
Y_pred_mlp_v2 = img_mlp_v2.predict(img_X_test)

Y_pred_svc_v1 = img_svc_v1.predict(img_X_test)
Y_pred_svc_v2 =  img_svc_v2.predict(img_X_test)

### Resultados

Observamos los resultados de las predicciones de los modelos y comparamos con los verdaderos resultados

|Modelo|Score|Cross Validation|
|:---|:---|:---|
|Random forest v1|78.78%|74.42%
|Random forest v2|78.24%|74.91%
|Neural Network v1|90.36%|87.99%
|Neural Network v2|90.08%|86.34%
|SVM v1|90.08%|85.76%
|SVM v2|90.08%|85.76%

In [19]:
print("Random forest v1 score:\t" , accuracy_score(img_Y_test, Y_pred_forest_v1))
print("Random forest v2 score:\t" , accuracy_score(img_Y_test, Y_pred_forest_v2))
print("Neural network v1 score:" , accuracy_score(img_Y_test, Y_pred_mlp_v1))
print("Neural network v2 score:" , accuracy_score(img_Y_test, Y_pred_mlp_v2))
print("SVC v1 score:\t\t" , accuracy_score(img_Y_test, Y_pred_svc_v1))
print("SVC v2 score:\t\t" , accuracy_score(img_Y_test, Y_pred_svc_v2))

Random forest v1 score:	 0.7878787878787878
Random forest v2 score:	 0.7823691460055097
Neural network v1 score: 0.9035812672176309
Neural network v2 score: 0.9008264462809917
SVC v1 score:		 0.9008264462809917
SVC v2 score:		 0.9008264462809917


#### Random Forest

In [20]:
cm_forest_v1 = metrics.confusion_matrix(img_Y_test, Y_pred_forest_v1)
print('Random Forest model 1')
print('Cross Validation:\n',cv(img_forest_v1, img_X, img_Y, cv=3, scoring='accuracy'))
print(cm_forest_v1)
print('\n')

Random Forest model 1
Cross Validation:
 [0.74193548 0.77915633 0.71144279]
[[51  0  2  0  0  0  0  0  0]
 [ 1 34  1  0  1  0  5  0  0]
 [ 0  2 37  1  0  0  3  0  0]
 [ 0  1  0 36  0  0  3  0  0]
 [ 0  2  1  0 38  0  3  1  0]
 [ 7  1  4  0  1 22  2  0  0]
 [ 3  0  0  1  1  1 46  0  0]
 [ 0  3  1  3  5  2  2  8  0]
 [ 3  4  1  0  2  2  1  0 14]]




In [21]:
cm_forest_v2 = metrics.confusion_matrix(img_Y_test, Y_pred_forest_v2)
print('Random Forest model 2')
print('Cross Validation:\n',cv(img_forest_v2, img_X, img_Y, cv=3, scoring='accuracy'))
print(cm_forest_v2)
print('\n')

Random Forest model 2
Cross Validation:
 [0.74937965 0.78411911 0.71393035]
[[51  0  0  0  0  1  1  0  0]
 [ 1 32  2  0  2  1  4  0  0]
 [ 1  2 36  1  0  0  3  0  0]
 [ 0  2  0 36  1  0  1  0  0]
 [ 0  2  1  0 39  0  2  1  0]
 [ 7  0  5  1  0 23  1  0  0]
 [ 2  0  0  0  2  1 47  0  0]
 [ 0  3  2  2  5  2  2  8  0]
 [ 3  4  1  0  3  2  2  0 12]]




#### SVM

In [22]:
cm_svc_v1 = metrics.confusion_matrix(img_Y_test, Y_pred_svc_v1)
print('SVM model 1')
print('Cross Validation:\n',cv(img_svc_v1, img_X, img_Y, cv=3, scoring='accuracy'))
print(cm_svc_v1)
print('\n')

SVM model 1
Cross Validation:
 [0.87096774 0.87593052 0.82587065]
[[49  0  2  0  0  1  1  0  0]
 [ 0 40  0  0  0  1  0  0  1]
 [ 0  2 40  0  0  0  1  0  0]
 [ 1  0  0 39  0  0  0  0  0]
 [ 0  2  1  0 40  0  1  1  0]
 [ 1  1  2  0  0 32  1  0  0]
 [ 1  1  0  1  2  2 45  0  0]
 [ 0  1  1  0  0  3  0 19  0]
 [ 0  1  1  0  1  1  0  0 23]]




In [23]:
cm_svc_v2 = metrics.confusion_matrix(img_Y_test, Y_pred_svc_v2)
print('SVM model 2')
print('Cross Validation:\n',cv(img_svc_v2, img_X, img_Y, cv=3, scoring='accuracy'))
print(cm_svc_v2)
print('\n')

SVM model 2
Cross Validation:
 [0.87096774 0.87593052 0.82587065]
[[49  0  2  0  0  1  1  0  0]
 [ 0 40  0  0  0  1  0  0  1]
 [ 0  2 40  0  0  0  1  0  0]
 [ 1  0  0 39  0  0  0  0  0]
 [ 0  2  1  0 40  0  1  1  0]
 [ 1  1  2  0  0 32  1  0  0]
 [ 1  1  0  1  2  2 45  0  0]
 [ 0  1  1  0  0  3  0 19  0]
 [ 0  1  1  0  1  1  0  0 23]]




#### Neural Network

In [24]:
cm_mlp_v1 = metrics.confusion_matrix(img_Y_test, Y_pred_mlp_v1)
print('Neural Network model 1')
print('Cross Validation:\n',cv(img_mlp_v1, img_X, img_Y, cv=3, scoring='accuracy'))
print(cm_mlp_v1)
print('\n')

Neural Network model 1
Cross Validation:
 [0.88585608 0.90074442 0.85323383]
[[49  0  1  0  0  1  1  0  1]
 [ 0 40  0  0  0  1  0  0  1]
 [ 3  1 37  0  0  1  1  0  0]
 [ 1  0  0 39  0  0  0  0  0]
 [ 0  1  1  0 39  0  0  4  0]
 [ 3  1  0  0  0 33  0  0  0]
 [ 2  1  1  1  2  0 45  0  0]
 [ 0  1  0  0  0  2  0 21  0]
 [ 0  1  0  0  0  1  0  0 25]]




In [25]:
cm_mlp_v2 = metrics.confusion_matrix(img_Y_test, Y_pred_mlp_v2)
print('Neural Network model 2')
print('Cross Validation:\n',cv(img_mlp_v2, img_X, img_Y, cv=3, scoring='accuracy'))
print(cm_mlp_v2)
print('\n')

Neural Network model 2
Cross Validation:
 [0.87593052 0.89081886 0.82338308]
[[50  0  0  0  0  1  1  0  1]
 [ 1 39  0  0  0  1  0  0  1]
 [ 2  1 37  0  0  1  1  0  1]
 [ 0  0  0 39  0  1  0  0  0]
 [ 0  1  0  1 38  0  1  4  0]
 [ 2  0  1  0  0 34  0  0  0]
 [ 1  0  1  1  3  0 46  0  0]
 [ 0  2  0  0  0  2  0 19  1]
 [ 0  1  1  0  0  0  0  0 25]]




### Conclusiones

A partir de los resultados que podemos observar con el score y el cross validation, podemos llegar a la conclusión de que los modelos generados son bastante confiables pues su índice de aciertos es bastante alto, lo suficiente como para basar la categorización de personajes marvel. Los modelos de random forest obtuvieron resultados son los mas bajos del grupo pero aún así siguen siendo buenos modelos para poder tomar en cuenta dentro de la categorización de los personajes.

Para poder obtener modelos que tengan resultados realmente buenos es necesario hacer varias pruebas o utilziar la herramienta del grid search, pues los modelos que se crean con los parametros predeterminados no siempre son los mejores, y en algunos casos podrían llegar a se aquellos con los peores resultados.